In [1]:
from mycode.test_20_tables import fetch_data_from_sqlite, Metadata as sdvMetadata
from sdgx.data_connectors.csv_connector import CsvConnector
from sdgx.models.ml.single_table.ctgan import CTGANSynthesizerModel
from sdgx.synthesizer import Synthesizer
import time
def writetime():
    st = time.time()
    with open("time.log", "a+") as f:
        f.write(str(st) + "\n")

构造数据

In [2]:
metadata, tables = fetch_data_from_sqlite(path='./mycode/data_sqlite.db')
metadata = sdvMetadata(metadata)
metadata.get_tables()
import pandas as pd
result_table = tables["BookLoan"]
x_table = ["Book", "Library", "Student", "Enrollment", "Submission"]
x_key = ['book_id', "library_id", "student_id", "student_id","student_id"]
x_how = ['inner' for i in range(5)]
print(f"{len(result_table)=}")
for i, x_table_name in enumerate(x_table):
    result_table = pd.merge(result_table, tables[x_table_name], on=x_key[i], how=x_how[i])
    print(f"{len(result_table)}", end=" ")
print(len(result_table.columns))
result_table

len(result_table)=10000
10000 10000 10000 9999 10798 23


,loan_id,book_id,student_id,loan_date,return_date,title,author,isbn,library_id,name_x,...,date_of_birth,major,enrollment_id,course_id,semester,grade_x,submission_id,assignment_id,submission_date,grade_y
0,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,5758,9618,Fall 2022,C,2462,9937,2022-10-18 02:02:31,B
1,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,5758,9618,Fall 2022,C,3984,4403,2022-11-15 20:13:23,I
2,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,5758,9618,Fall 2022,C,7851,6252,2022-07-31 08:44:12,A
3,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,9382,5777,Summer 2023,A,2462,9937,2022-10-18 02:02:31,B
4,1,1,5068,2024-07-19,2025-07-30,Model bank.,Ricky Brown,978-1-4017-1762-9,1,Wilson Group Library,...,2003-01-27,1262,9382,5777,Summer 2023,A,3984,4403,2022-11-15 20:13:23,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10793,9998,9998,3486,2024-01-14,2024-12-09,Listen interview world.,Richard Floyd,978-0-7738-9332-0,9998,Johnston Inc Library,...,2000-08-21,8143,3450,6477,Spring 2021,F,8054,1227,2022-09-20 01:25:41,C
10794,9998,9998,3486,2024-01-14,2024-12-09,Listen interview world.,Richard Floyd,978-0-7738-9332-0,9998,Johnston Inc Library,...,2000-08-21,8143,3757,4480,Fall 2019,None,578,6739,2022-11-01 16:39:13,A
10795,9998,9998,3486,2024-01-14,2024-12-09,Listen interview world.,Richard Floyd,978-0-7738-9332-0,9998,Johnston Inc Library,...,2000-08-21,8143,3757,4480,Fall 2019,None,8054,1227,2022-09-20 01:25:41,C
10796,9999,9999,2281,2023-12-28,2025-08-23,Challenge last suffer.,Robert Holland,978-1-67998-842-4,9999,Williams-Smith Library,...,1996-11-30,1005,9064,7483,Fall 2021,B,9247,1546,2022-08-05 15:48:08,B


In [3]:
result_table = result_table.drop(columns=["title", "author", "name_x", "semester", "location", "name_y", "email"])

In [4]:
result_table = result_table.drop(columns=["isbn"])

In [5]:
# result_table = result_table.drop(columns=["grade_x", "grade_y"])

In [6]:
# result_table.head()

In [7]:
result_table = result_table[:100]

检查metadata

In [8]:
from sdgx.data_loader import DataLoader
from sdgx.data_models.metadata import Metadata

result_table.to_csv("test_10k_noncat.csv", index=False)
dataset_csv = "test_10k_noncat.csv"
data_connector = CsvConnector(path=dataset_csv)
data_loader = DataLoader(data_connector)
loan_metadata = Metadata.from_dataloader(data_loader)

loan_metadata.discrete_columns

2024-09-28 16:29:52.212 | INFO     | sdgx.data_models.metadata:from_dataloader:291 - Inspecting metadata...
2024-09-28 16:29:52.291 | INFO     | sdgx.data_models.metadata:update_primary_key:500 - Primary Key updated: set().


{'date_of_birth',
 'grade_x',
 'grade_y',
 'loan_date',
 'return_date',
 'submission_date'}

In [9]:
loan_metadata.datetime_columns
loan_metadata.column_list.__len__()

15

训练

In [ ]:


ctgan = CTGANSynthesizerModel(
        epochs=1,
        batch_size=500,
        # device="cpu"
    )
# Initialize synthesizer, use CTGAN model
synthesizer = Synthesizer(
    model=ctgan,  # For quick demo
    data_connector=data_connector,
)


writetime()
synthesizer.fit()
writetime()
#synthesizer.save("test_model_10k_id10")

2024-09-28 16:40:41.426 | INFO     | sdgx.synthesizer:__init__:106 - Using data processors: ['nonvaluetransformer', 'outliertransformer', 'emailgenerator', 'chnpiigenerator', 'intvalueformatter', 'datetimeformatter', 'constvaluetransformer', 'emptytransformer', 'columnordertransformer']
2024-09-28 16:40:41.427 | INFO     | sdgx.data_models.metadata:from_dataloader:291 - Inspecting metadata...
2024-09-28 16:40:41.577 | INFO     | sdgx.data_models.metadata:update_primary_key:500 - Primary Key updated: set().
2024-09-28 16:40:41.578 | INFO     | sdgx.synthesizer:fit:296 - Fitting data processors...
2024-09-28 16:40:41.592 | INFO     | sdgx.data_processors.transformers.nan:fit:71 - NonValueTransformer Fitted.
2024-09-28 16:40:41.593 | INFO     | sdgx.data_processors.transformers.nan:fit:87 - NonValueTransformer get int columns: {'library_id', 'book_id', 'course_id', 'student_id', 'submission_id', 'enrollment_id', 'assignment_id', 'loan_id', 'major'}.
2024-09-28 16:40:41.594 | INFO     | sd

{'primary_keys': set(), 'column_list': ['loan_id', 'book_id', 'student_id', 'loan_date', 'return_date', 'library_id', 'date_of_birth', 'major', 'enrollment_id', 'course_id', 'grade_x', 'submission_id', 'assignment_id', 'submission_date', 'grade_y'], 'column_inspect_level': {'email_columns': 30, 'english_name_columns': 40, 'datetime_columns': 20, 'china_mainland_postcode_columns': 20, 'chinese_name_columns': 40, 'const_columns': 80, 'bool_columns': 10, 'empty_columns': 90, 'china_mainland_mobile_phone_columns': 30, 'chinese_company_name_columns': 40, 'id_columns': 20, 'unified_social_credit_code_columns': 30, 'china_mainland_address_columns': 30, 'discrete_columns': 10, 'china_mainland_id_columns': 30, 'int_columns': 10, 'float_columns': 10}, 'pii_columns': set(), 'id_columns': set(), 'int_columns': {'library_id', 'book_id', 'course_id', 'student_id', 'submission_id', 'enrollment_id', 'assignment_id', 'loan_id', 'major'}, 'float_columns': set(), 'bool_columns': set(), 'discrete_columns'

2024-09-28 16:40:44.142 | INFO     | sdgx.synthesizer:fit:315 - Initializing processed data loader...
2024-09-28 16:40:44.147 | INFO     | sdgx.data_processors.transformers.nan:convert:110 - Converting data using NonValueTransformer...
2024-09-28 16:40:44.151 | INFO     | sdgx.data_processors.transformers.nan:convert:130 - Converting data using NonValueTransformer... Finished.
2024-09-28 16:40:44.152 | INFO     | sdgx.data_processors.transformers.outlier:convert:84 - Converting data using OutlierTransformer...
2024-09-28 16:40:44.155 | INFO     | sdgx.data_processors.transformers.outlier:convert:106 - Converting data using OutlierTransformer... Finished.
2024-09-28 16:40:44.156 | INFO     | sdgx.data_processors.formatters.int:convert:50 - Converting data using IntValueFormatter... Finished  (No Action).
2024-09-28 16:40:44.156 | INFO     | sdgx.data_processors.formatters.datetime:convert:89 - Converting data using DatetimeFormatter... Finished (No datetime columns).
2024-09-28 16:40:44

C:\Users\Elvin\Desktop\GitHub\synthetic-data-generator\.ndarry_cache


2024-09-28 16:40:53.506 | INFO     | sdgx.models.ml.single_table.ctgan:_pre_fit:253 - Sampling data.
2024-09-28 16:41:04.086 | INFO     | sdgx.models.ml.single_table.ctgan:_pre_fit:258 - Initialize Generator.
2024-09-28 16:41:04.088 | INFO     | sdgx.models.ml.single_table.ctgan:fit:223 - CTGAN prefit finished, start CTGAN training.
2024-09-28 16:41:04.089 | INFO     | sdgx.models.ml.single_table.ctgan:_fit:275 - Fit using data_size:100, data_dim: 44.
2024-09-28 16:41:10.341 | INFO     | sdgx.models.ml.single_table.ctgan:_fit:301 - Starting model training, epochs: 1


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
data_res = synthesizer.sample(1000)
data_res

2024-09-28 16:31:02.974 | INFO     | sdgx.synthesizer:sample:357 - Sampling...


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 